In [2]:
# read it in to inspect it
with open('/content/input (1).txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
print("length of the dataset" , len(text))

length of the dataset 1115394


In [4]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [5]:
chars = sorted(set(text))
stoi = {char: i for i, char in enumerate(chars)}
itos = {i: char for i, char in enumerate(chars)}

def encode(s):
    """Encoder function: Convert a string to a list of integers based on stoi mapping."""
    return [stoi[char] for char in s]

def decode(l):
    """Decoder function: Convert a list of integers back to a string based on itos mapping."""
    return ''.join(itos[index] for index in l)
encoded_text = encode("hii there")
decoded_text = decode(encoded_text)

print(encoded_text)
print(decoded_text)


[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [6]:
import torch

def encode_text_to_tensor(text, encode_func):
    """
    Encodes a given text using a provided encoding function and converts it into a PyTorch tensor.

    Parameters:
    - text (str): The text to be encoded.
    - encode_func (function): The function used to encode the text.

    Returns:
    - torch.Tensor: The encoded text as a PyTorch tensor.
    """
    encoded_text = encode_func(text)
    tensor = torch.tensor(encoded_text, dtype=torch.long)  #Making this into tensor
    return tensor

data = encode_text_to_tensor(text, encode)


print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [7]:
# Let's now split up the data into train and validation sets
n = int(0.85*len(data))
train_data = data[:n]
val_data = data[n:]

In [8]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [11]:
# here are all the unique characters that occur in this text
unique_chars = ''.join(sorted({char for char in text}))
vocab_size = len(unique_chars)
print(unique_chars)
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [12]:
# We are not feeding all the data directly into the model so but making a batch and feeding it batwise for memory management

import torch

torch.manual_seed(1337)
batch_size = 4



def get_batch(data, batch_size, block_size):
    """Generate a batch of data for inputs x and targets y."""
    ix = torch.randint(0, len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

def print_batch_details(xb, yb):
    """Print details of the input and target batches."""
    print('inputs:')
    print(xb.shape)
    print(xb)
    print('targets:')
    print(yb.shape)
    print(yb)
    print('----')

    for b in range(batch_size):
        for t in range(block_size):
            context = xb[b, :t+1].tolist()
            target = yb[b, t].item()
            print(f"when input is {context} the target: {target}")




xb, yb = get_batch(train_data, batch_size, block_size)
print_batch_details(xb, yb)


inputs:
torch.Size([4, 8])
tensor([[53, 61, 57, 10,  0, 20, 43,  1],
        [39, 41, 43, 42,  1, 58, 46, 43],
        [52, 41, 43,  8,  0,  0, 24, 17],
        [26, 33, 31, 10,  0, 25, 53, 57]])
targets:
torch.Size([4, 8])
tensor([[61, 57, 10,  0, 20, 43,  1, 58],
        [41, 43, 42,  1, 58, 46, 43,  1],
        [41, 43,  8,  0,  0, 24, 17, 27],
        [33, 31, 10,  0, 25, 53, 57, 58]])
----
when input is [53] the target: 61
when input is [53, 61] the target: 57
when input is [53, 61, 57] the target: 10
when input is [53, 61, 57, 10] the target: 0
when input is [53, 61, 57, 10, 0] the target: 20
when input is [53, 61, 57, 10, 0, 20] the target: 43
when input is [53, 61, 57, 10, 0, 20, 43] the target: 1
when input is [53, 61, 57, 10, 0, 20, 43, 1] the target: 58
when input is [39] the target: 41
when input is [39, 41] the target: 43
when input is [39, 41, 43] the target: 42
when input is [39, 41, 43, 42] the target: 1
when input is [39, 41, 43, 42, 1] the target: 58
when input is [39

In [13]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
            perplexity=None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
            probabilities = F.softmax(logits, dim=-1)
            log_probabilities = torch.log(probabilities)
            log_prob = torch.gather(log_probabilities,1,targets.unsqueeze(1)).squeeze(1)
            N = targets.size(0)
            perplexity = torch.exp(-torch.sum(log_prob)/N)

        return logits, loss,perplexity

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits,loss,perplexity = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits,loss,perpexity = m(xb, yb)
print(logits.shape)
print(loss)
print(perpexity)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([32, 65])
tensor(4.9670, grad_fn=<NllLossBackward0>)
tensor(143.5975, grad_fn=<ExpBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3
